<a href="https://colab.research.google.com/github/tumibytes/cis9650-project03/blob/main/term_project_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of the Mutuals Funds from Schwab

### Authors: Elise Tumibay, Dylan Lee, Creed Berges, Dianny Nunez
### Date : December 8th, 2025

## Executive Summary

## Table of Contents

1. Introduction
2. Problem Statement / Research Question
3. Data Description
4. Setup and Environment
5. Data Loading
6. Data Preparation
7. Model Planning
8. Model Building / Analysis
9. Discussion & Interpretation
10. Conclusion
11. References
12. Appendix

## Introduction

## Problem Statement / Research Question

## Data Description

## Setup and Environment

In [1]:
!pip install google-cloud-storage

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import os
from google.cloud import storage
from io import BytesIO

## Data Loading

### Functions

In [3]:
from google.cloud import storage

def list_public_bucket(bucket_name, course, project):
    prefix = f"{course}/Project {project}/"

    client = storage.Client.create_anonymous_client()
    bucket = client.bucket(bucket_name)
    blobs = client.list_blobs(bucket, prefix=prefix)
    print(f"Listing files in public bucket '{bucket_name}' under '{prefix}':")
    filenames = []
    for blob in blobs:
        print(blob.name)
        filenames.append(blob.name)

    return filenames

In [4]:
from google.cloud import storage
import pandas as pd
from io import BytesIO
import os
import json

def gcs_file_to_dataframe(bucket_name: str, blob_path: str):

    # Anonymous client for public buckets
    client = storage.Client.create_anonymous_client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_path)

    # Download file content as bytes
    data = blob.download_as_bytes()

    # Detect file extension
    _, ext = os.path.splitext(blob_path.lower())

    if ext == ".csv":
        df = pd.read_csv(BytesIO(data))

    elif ext in [".xls", ".xlsx"]:
        df = pd.read_excel(BytesIO(data))

    elif ext == ".parquet":
        df = pd.read_parquet(BytesIO(data))

    elif ext == ".json":
        # Convert bytes → Python object
        text = data.decode("utf-8")
        json_obj = json.loads(text)

        # If JSON is a list → NORMAL DataFrame
        if isinstance(json_obj, list):
            df = pd.DataFrame(json_obj)

        # If JSON is a dict → flatten into rows
        elif isinstance(json_obj, dict):
            df = pd.json_normalize(json_obj)

        else:
            raise ValueError("Unsupported JSON structure")

    else:
        raise ValueError(f"Unsupported file type: {ext}")

    return df

In [5]:
# Example
BUCKET_NAME = "msba-online-data"
PROJECT_NUMBER = "03"
COURSE = "CIS9650"
job_listings = list_public_bucket(BUCKET_NAME,COURSE,PROJECT_NUMBER)

Listing files in public bucket 'msba-online-data' under 'CIS9650/Project 03/':
CIS9650/Project 03/
CIS9650/Project 03/page_1.json
CIS9650/Project 03/page_10.json
CIS9650/Project 03/page_100.json
CIS9650/Project 03/page_101.json
CIS9650/Project 03/page_102.json
CIS9650/Project 03/page_103.json
CIS9650/Project 03/page_104.json
CIS9650/Project 03/page_105.json
CIS9650/Project 03/page_106.json
CIS9650/Project 03/page_107.json
CIS9650/Project 03/page_108.json
CIS9650/Project 03/page_109.json
CIS9650/Project 03/page_11.json
CIS9650/Project 03/page_110.json
CIS9650/Project 03/page_111.json
CIS9650/Project 03/page_112.json
CIS9650/Project 03/page_113.json
CIS9650/Project 03/page_114.json
CIS9650/Project 03/page_115.json
CIS9650/Project 03/page_116.json
CIS9650/Project 03/page_117.json
CIS9650/Project 03/page_118.json
CIS9650/Project 03/page_119.json
CIS9650/Project 03/page_12.json
CIS9650/Project 03/page_120.json
CIS9650/Project 03/page_121.json
CIS9650/Project 03/page_122.json
CIS9650/Project

In [6]:
# We'll need to revisit this part of the code
for element in job_listings[1:]:
    df = gcs_file_to_dataframe(BUCKET_NAME,  element)
    print(df.head())
    break

                                                data  meta.currentPage  \
0  [{'id': 'SCEA20FXJZvvUkfV', 'detailsPageUrl': ...                 1   

   meta.pageSize  meta.pageCount  meta.totalResults  meta.includeRemote  \
0            100             343              34212               False   

  meta.sortBy meta.facetQueryResults meta.searchedLocation.countryCode2  \
0    UNSORTED                     []                                 US   

  meta.searchedLocation.radiusUnit  meta.searchedLocation.radius  \
0                               km                           100   

                                meta.selectedFilters  \
0  [{'filterName': 'unspecifiedSalary', 'displayN...   

                          meta.searchId  meta.totalJobCount meta.topCompanies  \
0  cec47c71-1293-48ca-a35c-53402a9b0527               34212                []   

                                    _links.self.href  \
0  /v1/efc/jobs/search?page=1&pageSize=100&search...   

                       

In [10]:
df_expanded = df.explode("data", ignore_index=True)

# 2. Convert the dict into columns
detail_cols = df_expanded["data"].apply(pd.Series)

# 3. Merge back and drop the dict column
df_flat = pd.concat([df_expanded.drop(columns=["data"]), detail_cols], axis=1)

df_flat

,meta.currentPage,meta.pageSize,meta.pageCount,meta.totalResults,meta.includeRemote,meta.sortBy,meta.facetQueryResults,meta.searchedLocation.countryCode2,meta.searchedLocation.radiusUnit,meta.searchedLocation.radius,...,locationId,cityId,countryId,language,minSalary,maxSalary,salaryCurrency,jobPaymentType,fullCompanyName,workArrangementType
0,1,100,343,34212,False,UNSORTED,[],US,km,100,...,4843564,4843564,6252001,EN,0,0,USD,PAID,Axa Investment Managers Paris,NaN
1,1,100,343,34212,False,UNSORTED,[],US,km,100,...,5128581,5128581,6252001,EN,83000,137000,USD,PAID,Deutsche Bank US,Hybrid
2,1,100,343,34212,False,UNSORTED,[],US,km,100,...,4459467,4459467,6252001,EN,100000,142250,USD,PAID,Deutsche Bank US,Hybrid
3,1,100,343,34212,False,UNSORTED,[],US,km,100,...,5045360,5045360,6252001,EN,84000,127000,USD,PAID,QBE Insurance (Europe) Limited,Remote
4,1,100,343,34212,False,UNSORTED,[],US,km,100,...,5275020,5275020,6252001,EN,100000,150000,USD,PAID,QBE Insurance (Europe) Limited,Remote
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,100,343,34212,False,UNSORTED,[],US,km,100,...,5128581,5128581,6252001,EN,0,0,USD,UNPAID,ASPEN - Capital One,NaN
96,1,100,343,34212,False,UNSORTED,[],US,km,100,...,4781708,4781708,6252001,EN,0,0,USD,UNPAID,ASPEN - Capital One,NaN
97,1,100,343,34212,False,UNSORTED,[],US,km,100,...,5391959,5391959,6252001,EN,0,0,USD,UNPAID,ASPEN - Capital One,NaN
98,1,100,343,34212,False,UNSORTED,[],US,km,100,...,4560349,4560349,6252001,EN,0,0,USD,UNPAID,RADANCY - Citizens,NaN


## Data Preparation

## Model Planning

## Model building / Analysis

## Discussion and Results

## Conclusion

## References

1. Barber, David. Bayesian Reasoning and Machine Learning. Cambridge University Press, 2012.
2. Aste, Tomaso, Paola Cerchiello, and Roberta Scaramozzino. "Information-Theoretic Causality Detection between Financial and Sentiment Data."Entropy, vol. 24, no. 6, 2022, pp. 1–18. DOI:10.3390/e24060774.
3. Metz, Cade. "Microsoft Puts OpenAI’s Sam Altman in Charge of New Advanced AI Research Team."
The New York Times, 20 Nov. 2023, www.nytimes.com/2023/11/20/technology/openai-microsoft-altman.html

## Appendix